# Validate On Anomalies

In [ ]:
import numpy as np
import pandas as pd

import os
import sys
from pathlib import Path
SCRIPT_DIR = os.path.dirname(os.path.abspath("__init__.py"))
SRC_DIR = Path(SCRIPT_DIR).parent.absolute()
print(SRC_DIR)
sys.path.append(os.path.dirname(SRC_DIR))

from src.config import TA_MU, TA_SIGMA, SLP_MU, SLP_SIGMA

# Compare Normalized Anomaly MAEs 

In [ ]:
EXPERIMENTS = [
               'ex1_baseline',  # Is already anomaly 
               'ex2_seasonal_component',
               'ex3_incremental_pretraining',
               'ex3.1_moving_window',
               'ex3.2_cm_inclusion',
               'ex3.3_elevation',
               'ex3.4_pi_init',
               'ex4.1_elev_mov_win',
               'ex4.2_elev_cmi',
               'ex4.3_elev_pi_init',
               'ex5.1_elev_mov_cmi',
               'ex5.2_elev_mov_pi',
               'ex6.1_elev_mov_cmi_pi'
               ]

In [ ]:
# Unit refers to °C or Pa
unit_val_ymean = np.load("../data_sets/ymean_sets/validation_ymean.npy")

normalized_gt = np.load("../data_sets/variant_b/validation.npy")
normalized_gt = normalized_gt[:3644, 6:38, 2:66]

normalized_val_ymean = (unit_val_ymean - np.array([TA_MU, SLP_MU])) / np.array([TA_SIGMA, SLP_SIGMA])
normalized_val_ymean = normalized_val_ymean[:3644]

exp_mae_map = {}
for exp_name in EXPERIMENTS:
    normalized_pred = np.load(f"{exp_name}/prediction/predicted.npy")

    if exp_name == "ex1_baseline":
        anom_gt = np.load("../data_sets/variant_a/validation.npy")
        anom_gt = anom_gt[:3644, 6:38, 2:66]
        _mae = np.mean(np.abs(normalized_pred - anom_gt))
        print(f"Experiment {exp_name}: {_mae}")
        exp_mae_map[exp_name] = _mae
        continue
    
    norm_gt_anom = normalized_gt - normalized_val_ymean
    norm_pred_anom = normalized_pred - normalized_val_ymean
    
    normalized_anomaly_mae = np.mean(np.abs(norm_pred_anom - norm_gt_anom)) 
    print(f"Experiment {exp_name}: {normalized_anomaly_mae}")
    exp_mae_map[exp_name] = normalized_anomaly_mae

In [ ]:
# Almost IDENTICAL to the climatology-MAE results! -> Seasonal component has been learned very well!
exp_mae_map

In [ ]:
df = pd.DataFrame(data={'exp_name': exp_mae_map.keys(), 'anomaly_mae': exp_mae_map.values()} )
df.round(4)

## Compare Normalized Anomaly RMSEs 

In [ ]:
def calc_rmse(gt, pred):
    squared_diffs = np.square(gt - pred)
    # Calculate the mean of the squared differences
    mean_squared_diff = np.mean(squared_diffs)
    # Calculate the square root of the mean squared differences (RMSE)
    return np.sqrt(mean_squared_diff)

In [ ]:
# Unit refers to °C or Pa
unit_val_ymean = np.load("../data_sets/ymean_sets/validation_ymean.npy")

normalized_gt = np.load("../data_sets/variant_b/validation.npy")
normalized_gt = normalized_gt[:3644, 6:38, 2:66]

normalized_val_ymean = (unit_val_ymean - np.array([TA_MU, SLP_MU])) / np.array([TA_SIGMA, SLP_SIGMA])
normalized_val_ymean = normalized_val_ymean[:3644]

norm_gt_anom = normalized_gt - normalized_val_ymean

exp_rmse_map = {}
for exp_name in EXPERIMENTS:
    normalized_pred = np.load(f"{exp_name}/prediction/predicted.npy")
    if exp_name == "ex1_baseline":
        anom_gt = np.load("../data_sets/variant_a/validation.npy")
        anom_gt = anom_gt[:3644, 6:38, 2:66]
        _rmse = calc_rmse(anom_gt, normalized_pred)
        print(f"Experiment {exp_name}: {_rmse}")
        exp_rmse_map[exp_name] = _rmse
        continue
    
    norm_pred_anom = normalized_pred - normalized_val_ymean
    
    normalized_anomaly_rmse = calc_rmse(norm_gt_anom, norm_pred_anom) 
    print(f"Experiment {exp_name}: {normalized_anomaly_rmse}")
    exp_rmse_map[exp_name] = normalized_anomaly_rmse

In [ ]:
df_rmse = pd.DataFrame(data={'exp_name': exp_rmse_map.keys(), 'anomaly_rmse': exp_rmse_map.values()} )
df_rmse.round(4)

In [ ]:
df['anomaly_rmse'] = df_rmse['anomaly_rmse']

In [ ]:
df.round(4)